### **1. Setup**

In [2]:
import os, requests, sys, json
import numpy as np
import pandas as pd
from time import sleep

# https://rest.uniprot.org/beta/docs/
WEBSITE_API = "https://rest.uniprot.org/beta"

# Helper function to download data
def get_url(url, **kwargs):
  response = requests.get(url, **kwargs);

  if not response.ok:
    print(response.text)
    response.raise_for_status()
    sys.exit()

  return response

In [1]:
# directory
directory = '/Users/ShokenLEE/Desktop/DATA/Bioinformatics/\
MemBrain AH prediction/'

### File import

In [3]:
os.chdir(directory)

In [4]:
os.getcwd()

'/Users/ShokenLEE/Desktop/DATA/Bioinformatics/MemBrain AH prediction'

In [5]:
filename = 'Result_Selected_Organisms.csv'

df = pd.read_csv(directory + filename)

In [7]:
df['Organism']==''.head(n=10)

,Uniprot_ID,Organism,Protein_name,AH_or_Not,AA_sequence,Prediction
0,Q8N4K4,Homo sapiens (Human),Reprimo-like protein,Non-AH,MNATFLNHSGLEEVDGVGGGAGAALGNRTHGLGTWLGCCPGGAPLA...,0000000000000000000000000000000000000000000000...
1,Q8N4S7,Homo sapiens (Human),Progestin and adipoQ receptor family member 4,Non-AH,MAFLAGPRLLDWASSPPHLQFNKFVLTGYRPASSGSGCLRSLFYLH...,0000000000000000000000000000000000000000000000...
2,Q8N5G0,Homo sapiens (Human),Small integral membrane protein 20,Non-AH,MSRNLRTALIFGGFISLIGAAFYPIYFRPLMRLEEYKKEQAINRAG...,0000000000000000000000000000000000000000000000...
3,Q8N614,Homo sapiens (Human),Transmembrane protein 156,AH,MTKTALLKLFVAIVITFILILPEYFKTPKERTLELSCLEVCLQSNF...,0011111100000000000000000000000000000000000000...
4,Q8N7C4,Homo sapiens (Human),Transmembrane protein 217,Non-AH,MKQQQWCGMTAKMGTVLSGVFTIMAVDMYLIFEQKHLGNGSCTEIT...,0000000000000000000000000000000000000000000000...
5,Q8N7S6,Homo sapiens (Human),Uncharacterized protein ARIH2OS,Non-AH,MLGQRAGDGERPGLPGDGEGGVPARPGRRAERPPQRPAKVNKAVTC...,0000000000000000000000000000000000000000000000...
6,Q8N7X8,Homo sapiens (Human),SIGLEC family-like protein 1,Non-AH,MLPLLQLVPAKLLNSSCSLEKTLQCSCSFHGIPTPSVQWWMGGVPV...,0000000000000000000000000000000000000000000000...
7,Q8N816,Homo sapiens (Human),Uncharacterized protein KRT10-AS1,Non-AH,MVGILPLCCSGCVPSLCCSSYVPSVAPTAAHSVRVPHSAGHCGQRV...,0000000000000000000000000000000000000000000000...
8,Q8N8F6,Homo sapiens (Human),Protein YIPF7,Non-AH,MDLLKISHTKLHLLEDLSIKNKQRMSNLAQFDSDFYQSNFTIDNQE...,0000000000000000000000000000000000000000000000...
9,Q8N8F7,Homo sapiens (Human),Leucine-rich single-pass membrane protein 1,AH,MTHSSQDTGSCGIQEDGKLYVVDSINDLNKLNLCPAGSQHLFPLED...,0000000000000000000000000000000000000000000000...


### **2. Examine**

In [6]:
df.shape

(2992, 6)

In [63]:
proteins = list(df['Protein_name'])
proteins_unique = list(df['Protein_name'].unique())
print('The total number of protein names: ', len(proteins))
print('The total number of unique protein names: ', len(proteins_unique))

The total number of protein names:  2992
The total number of unique protein names:  2892


In [79]:
duplicated_proteins = [protein for protein in proteins_unique 
                       if proteins.count(protein) >= 2]
print('The number of the duplicated_proteins: ', len(duplicated_proteins))

The number of the duplicated_proteins:  93


##### Note that there are 3 and 2 of 3 and 4 times duplicated proteins
##### this makes the subtraction of the total minus unique 100

### **3. Subcellular locactions**

#### 01-10-22 From 0 to 299

In [8]:
for i, uniprot_id in enumerate(df['Uniprot_ID'][:300]):
    # get the data as a json
    try:
        r = get_url(f"{WEBSITE_API}/uniprotkb/{uniprot_id}?fields=cc_subcellular_location")
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        
    data = r.json()
    
    # get the subcellular location
    subcellular_location = []
    for location_and_topology in data['comments'][0]['subcellularLocations']:
        subcellular_location.append(location_and_topology['location']['value'])
    
    # put into df as a joined string
    joined = ', '.join(subcellular_location)
    df.loc[i, 'Subcellular_location'] = joined
    
    # log
    print(i, uniprot_id, joined)
    
    # sleep
    sleep(1)

0 Q8N4K4 Membrane
1 Q8N4S7 Membrane
2 Q8N5G0 Mitochondrion inner membrane
3 Q8N614 Membrane
4 Q8N7C4 Membrane
5 Q8N7S6 Membrane
6 Q8N7X8 Membrane
7 Q8N816 Membrane
8 Q8N8F6 Endoplasmic reticulum membrane, Golgi apparatus, cis-Golgi network membrane, Golgi apparatus, trans-Golgi network membrane
9 Q8N8F7 Membrane
10 Q8N8V8 Membrane
11 Q8N8Z6 Membrane
12 Q8N9F0 Cytoplasm, Membrane, Microsome membrane, Mitochondrion membrane, Rough endoplasmic reticulum membrane
13 Q8N9I5 Membrane
14 Q8N9R8 Membrane, Nucleus, Cytoplasm
15 Q8N9X5 Membrane
16 Q8NA29 Cell membrane, Endoplasmic reticulum membrane
17 Q8NAC3 Cell membrane
18 Q8NAN2 Mitochondrion outer membrane
19 Q8NBP5 Membrane
20 Q8NBR0 Cell membrane, Cytoplasm
21 Q8NBS3 Cell membrane, Basolateral cell membrane
22 Q8NC44 Membrane
23 Q8NCG7 Cell membrane
24 Q8NCK7 Endoplasmic reticulum membrane, Cell membrane
25 Q8NCQ3 Membrane
26 Q8NCS4 Membrane
27 Q8NDB6 Membrane
28 Q8NDY8 Membrane
29 Q8NEA5 Membrane
30 Q8NEQ5 Membrane
31 Q8NET5 Cell membran

In [9]:
df.to_csv('./Result_Selected_Organisms_SubCellLoc.csv', index=False)

#### 01-11-22 From 300 to 599

In [15]:
filename = 'Result_Selected_Organisms_SubCellLoc.csv'

df_WIP = pd.read_csv(directory + filename)

df_WIP = df_WIP.drop(['Unnamed: 0'], axis=1)

In [26]:
index_start = 300
for i in range(index_start, index_start + 300):
    # get the uniprot id
    uniprot_id = df_WIP['Uniprot_ID'][i]
    
    # get the data as a json
    try:
        r = get_url(f"{WEBSITE_API}/uniprotkb/{uniprot_id}?fields=cc_subcellular_location")
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    
    data = r.json()
    
    
    # get the subcellular location
    subcellular_location = []
    for location_and_topology in data['comments'][0]['subcellularLocations']:
        subcellular_location.append(location_and_topology['location']['value'])
    
    # put into df as a joined string
    joined = ', '.join(subcellular_location)
    df_WIP.loc[i, 'Subcellular_location'] = joined
    
    # log
    print(i, uniprot_id, joined)
    
    # sleep
    sleep(1)

300 Q96RD9 Cell membrane
301 Q96RL6 Membrane
302 Q96S66 Endoplasmic reticulum membrane, Golgi apparatus membrane, Nucleus membrane
303 Q96T54 Membrane
304 Q99523 Golgi apparatus, Golgi stack membrane, Endosome membrane, Endoplasmic reticulum membrane, Nucleus membrane, Cell membrane, Lysosome membrane
305 Q99650 Membrane
306 Q99665 Membrane
307 Q99795 Membrane
308 Q99J93 Cell membrane, Lysosome membrane, Late endosome membrane
309 Q99K24 Membrane
310 Q99LJ5 Membrane
311 Q99LY2 Membrane, Cytoplasm, perinuclear region, Endoplasmic reticulum membrane
312 Q99M08 Membrane
313 Q99MB1 Endoplasmic reticulum membrane, Endosome membrane, Early endosome
314 Q99MB3 Membrane
315 Q99N10 Membrane
316 Q99P91 Cell membrane, Melanosome membrane, Early endosome membrane
317 Q99PE5 Membrane
318 Q9ASQ2 Endoplasmic reticulum membrane, Nucleus
319 Q9AST6 Endosome membrane
320 Q9ASU1 Endoplasmic reticulum membrane, Lipid droplet
321 Q9BPN8 Membrane
322 Q9BQG1 Cell membrane, Cytoplasmic vesicle, secretory vesi

#### 01-11-22 From 600 to 899

In [28]:
index_start = 600
for i in range(index_start, index_start + 300):
    # get the uniprot id
    uniprot_id = df_WIP['Uniprot_ID'][i]
    
    # get the data as a json
    try:
        r = get_url(f"{WEBSITE_API}/uniprotkb/{uniprot_id}?fields=cc_subcellular_location")
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    
    data = r.json()
    
    
    # get the subcellular location
    subcellular_location = []
    for location_and_topology in data['comments'][0]['subcellularLocations']:
        subcellular_location.append(location_and_topology['location']['value'])
    
    # put into df as a joined string
    joined = ', '.join(subcellular_location)
    df_WIP.loc[i, 'Subcellular_location'] = joined
    
    # log
    print(i, uniprot_id, joined)
    
    # sleep
    sleep(1)

600 Q9LYR6 Membrane
601 Q9LYU7 Cell membrane, Endoplasmic reticulum membrane
602 Q9LZ31 Membrane
603 Q9LZA6 Lipid droplet, Membrane, Peroxisome
604 Q9LZB8 Plastid, chloroplast membrane
605 Q9LZD0 Plastid, chloroplast envelope, Plastid, chloroplast membrane
606 Q9LZH8 Plastid, chloroplast outer membrane
607 Q9LZJ6 Membrane
608 Q9LZM4 Membrane
609 Q9LZR8 Membrane
610 Q9LZT4 Membrane
611 Q9M008 Membrane
612 Q9M012 Endosome membrane
613 Q9M066 Endoplasmic reticulum membrane
614 Q9M0G0 Vacuole membrane
615 Q9M0R4 Membrane
616 Q9M0Y1 Cell membrane
617 Q9M0Z3 Plastid, chloroplast membrane, Golgi apparatus membrane, Plastid, chloroplast thylakoid membrane
618 Q9M115 Endoplasmic reticulum membrane, Cytoplasmic vesicle membrane
619 Q9M130 Membrane
620 Q9M1B5 Membrane
621 Q9M1J0 Membrane
622 Q9M1J1 Mitochondrion outer membrane, Peroxisome membrane
623 Q9M1K5 Cell membrane
624 Q9M2G7 Endoplasmic reticulum membrane
625 Q9M2S6 Endoplasmic reticulum membrane
626 Q9M2Y6 Golgi apparatus membrane
627 Q9

#### 01-11-22 From 900 to 1199

In [31]:
index_start = 900
for i in range(index_start, index_start + 300):
    # get the uniprot id
    uniprot_id = df_WIP['Uniprot_ID'][i]
    
    # get the data as a json
    try:
        r = get_url(f"{WEBSITE_API}/uniprotkb/{uniprot_id}?fields=cc_subcellular_location")
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    
    data = r.json()
    
    
    # get the subcellular location
    subcellular_location = []
    for location_and_topology in data['comments'][0]['subcellularLocations']:
        subcellular_location.append(location_and_topology['location']['value'])
    
    # put into df as a joined string
    joined = ', '.join(subcellular_location)
    df_WIP.loc[i, 'Subcellular_location'] = joined
    
    # log
    print(i, uniprot_id, joined)
    
    # sleep
    sleep(1)

900 Q9WUP0 Membrane
901 Q9WVK8 Endoplasmic reticulum membrane, Microsome membrane, Cell junction, synapse, postsynapse, Cell junction, synapse, presynapse, Cell projection, dendrite
902 Q9XEC8 Membrane
903 Q9XF63 Membrane
904 Q9XFR0 Vacuole membrane
905 Q9XI46 Mitochondrion inner membrane
906 Q9XIA7 Mitochondrion outer membrane
907 Q9XID2 Cell membrane
908 Q9XII1 Plastid, chloroplast outer membrane
909 Q9XIN1 Membrane
910 Q9XT97 Endoplasmic reticulum, Endoplasmic reticulum membrane, Golgi apparatus membrane, Cytoplasmic granule, Cell membrane, Cell projection, growth cone, Early endosome, Early endosome membrane, Cell projection, neuron projection, Cell projection, axon, Cell junction, synapse
911 Q9XTG1 Cell membrane, Cell junction, synapse
912 Q9XU42 Cell membrane
913 Q9XU63 Cell membrane
914 Q9XU98 Cell membrane
915 Q9XVM9 Membrane
916 Q9XVV5 Endoplasmic reticulum membrane
917 Q9XWE9 Membrane
918 Q9XWU2 Cell membrane, Perikaryon, Cell projection, Cell junction, synapse, Cytoplasmic 

#### 01-11-22 From 1200 to 1499

In [32]:
index_start = 1200
for i in range(index_start, index_start + 300):
    # get the uniprot id
    uniprot_id = df_WIP['Uniprot_ID'][i]
    
    # get the data as a json
    try:
        r = get_url(f"{WEBSITE_API}/uniprotkb/{uniprot_id}?fields=cc_subcellular_location")
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    
    data = r.json()
    
    
    # get the subcellular location
    subcellular_location = []
    for location_and_topology in data['comments'][0]['subcellularLocations']:
        subcellular_location.append(location_and_topology['location']['value'])
    
    # put into df as a joined string
    joined = ', '.join(subcellular_location)
    df_WIP.loc[i, 'Subcellular_location'] = joined
    
    # log
    print(i, uniprot_id, joined)
    
    # sleep
    sleep(1)

1200 P19320 Membrane
1201 P19467 Cell membrane, Secreted
1202 P20036 Cell membrane, Endoplasmic reticulum membrane, Golgi apparatus, trans-Golgi network membrane, Endosome membrane, Lysosome membrane
1203 P20237 Cell junction, synapse, postsynaptic cell membrane, Cell membrane
1204 P20274 Membrane
1205 P20291 Nucleus membrane, Endoplasmic reticulum membrane
1206 P20334 Membrane
1207 P20350 Golgi apparatus membrane
1208 P20491 Cell membrane
1209 P20792 Membrane
1210 P20917 Cell membrane, Membrane raft
1211 P21995 Cell membrane, Cell junction, synapse
1212 P22794 Membrane
1213 P22815 Cell membrane
1214 P23645 Membrane
1215 P23913 Nucleus inner membrane, Nucleus, Cytoplasm, Endoplasmic reticulum membrane
1216 P24071 Cell membrane
1217 P24585 Membrane
1218 P24895 Mitochondrion membrane
1219 P25119 Membrane
1220 P25123 Cell membrane, Cell junction, synapse, postsynaptic cell membrane, Cell projection, dendrite, Cell projection, axon
1221 P25445 Cell membrane, Membrane raft
1222 P26012 Cell 

#### 01-11-22 From 1500 to 1799
##### From here try except was added to data comments

In [37]:
index_start = 1500
for i in range(index_start, index_start + 300):
    # get the uniprot id
    uniprot_id = df_WIP['Uniprot_ID'][i]
    
    # get the data as a json
    try:
        r = get_url(f"{WEBSITE_API}/uniprotkb/{uniprot_id}?fields=cc_subcellular_location")
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    
    data = r.json()
    
    
    # get the subcellular location
    try:
        subcellular_location = []
        for location_and_topology in data['comments'][0]['subcellularLocations']:
            subcellular_location.append(location_and_topology['location']['value'])
    except:
        subcellular_location = ['Not_found']
    
    # put into df as a joined string
    joined = ', '.join(subcellular_location)
    df_WIP.loc[i, 'Subcellular_location'] = joined
    
    # log
    print(i, uniprot_id, joined)
    
    # sleep
    sleep(1)

1500 Q09341 Membrane
1501 Q09344 Membrane
1502 Q09351 Membrane
1503 Q09356 Not_found
1504 Q09366 Membrane
1505 Q09368 Membrane
1506 Q09371 Membrane
1507 Q09372 Membrane
1508 Q09426 Membrane, Endoplasmic reticulum
1509 Q09488 Membrane
1510 Q09500 Lysosome membrane, Cytoplasmic vesicle, phagosome
1511 Q09562 Membrane
1512 Q09572 Membrane
1513 Q09606 Membrane
1514 Q09621 Membrane
1515 Q09630 Cell membrane
1516 Q09653 Membrane
1517 Q09951 Membrane
1518 Q09952 Membrane
1519 Q09954 Membrane
1520 Q09964 Cell membrane
1521 Q09966 Cell membrane
1522 Q09969 Mitochondrion outer membrane
1523 Q09993 Membrane
1524 Q0D285 Membrane
1525 Q0D289 Cell projection, cilium membrane, Cell membrane
1526 Q0D2G3 Membrane
1527 Q0D2K0 Membrane
1528 Q0EEE2 Membrane
1529 Q0GA42 Cell membrane
1530 Q0IIE5 Cell junction, adherens junction, Cell membrane
1531 Q0IIL4 Golgi apparatus membrane
1532 Q0P496 Golgi apparatus membrane
1533 Q0P4A4 Membrane
1534 Q0P5H8 Membrane
1535 Q0P5M9 Nucleus inner membrane
1536 Q0P5V9 Mem

#### 01-11-22 From 1800 to 2099

In [40]:
index_start = 1800
for i in range(index_start, index_start + 300):
    # get the uniprot id
    uniprot_id = df_WIP['Uniprot_ID'][i]
    
    # get the data as a json
    try:
        r = get_url(f"{WEBSITE_API}/uniprotkb/{uniprot_id}?fields=cc_subcellular_location")
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    
    data = r.json()
    
    
    # get the subcellular location
    try:
        subcellular_location = []
        for location_and_topology in data['comments'][0]['subcellularLocations']:
            subcellular_location.append(location_and_topology['location']['value'])
    except:
        subcellular_location = ['Not_found']
    
    # put into df as a joined string
    joined = ', '.join(subcellular_location)
    df_WIP.loc[i, 'Subcellular_location'] = joined
    
    # log
    print(i, uniprot_id, joined)
    
    # sleep
    sleep(1)

1800 Q38820 Mitochondrion inner membrane
1801 Q38833 Plastid, chloroplast membrane
1802 Q38906 Plastid, chloroplast outer membrane
1803 Q38944 Membrane
1804 Q38954 Plastid, chloroplast inner membrane
1805 Q38967 Cell membrane
1806 Q39017 Membrane
1807 Q39065 Membrane
1808 Q39079 Membrane
1809 Q39165 Lipid droplet, Membrane
1810 Q39208 Endoplasmic reticulum membrane
1811 Q39253 Vacuole membrane
1812 Q3B7S5 Membrane
1813 Q3B7T3 Membrane
1814 Q3B8R1 Endoplasmic reticulum membrane
1815 Q3C1V0 Membrane
1816 Q3E7D0 Cell membrane
1817 Q3E8L0 Membrane
1818 Q3E8Z8 Membrane
1819 Q3E9Z9 Membrane
1820 Q3EA54 Cell membrane
1821 Q3EAQ5 Membrane
1822 Q3EAV6 Membrane
1823 Q3EBC2 Endoplasmic reticulum membrane, Cytoplasmic vesicle membrane
1824 Q3EBR4 Cell membrane
1825 Q3ECH2 Membrane
1826 Q3ECI7 Mitochondrion outer membrane
1827 Q3EDG5 Golgi apparatus membrane
1828 Q3KR37 Endoplasmic reticulum membrane, Cell membrane
1829 Q3KRC4 Cell membrane
1830 Q3KTM0 Plastid, chloroplast membrane
1831 Q3LRV9 Cell

#### 01-11-22 From 2100 to end

In [41]:
df_WIP.shape

(2992, 7)

In [42]:
len(df_WIP)

2992

In [43]:
index_start = 2100
for i in range(index_start, len(df_WIP)):
    # get the uniprot id
    uniprot_id = df_WIP['Uniprot_ID'][i]
    
    # get the data as a json
    try:
        r = get_url(f"{WEBSITE_API}/uniprotkb/{uniprot_id}?fields=cc_subcellular_location")
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    
    data = r.json()
    
    
    # get the subcellular location
    try:
        subcellular_location = []
        for location_and_topology in data['comments'][0]['subcellularLocations']:
            subcellular_location.append(location_and_topology['location']['value'])
    except:
        subcellular_location = ['Not_found']
    
    # put into df as a joined string
    joined = ', '.join(subcellular_location)
    df_WIP.loc[i, 'Subcellular_location'] = joined
    
    # log
    print(i, uniprot_id, joined)
    
    # sleep
    sleep(1)

2100 Q68FR5 Membrane
2101 Q68FV4 Membrane
2102 Q69FB3 Cell membrane, Endoplasmic reticulum membrane
2103 Q69YZ2 Membrane
2104 Q69Z08 Cell projection, cilium membrane, Cell projection, cilium, Cytoplasm, cytoskeleton, cilium axoneme, Cytoplasm, cytoskeleton
2105 Q6AXS2 Membrane
2106 Q6AXT9 Membrane
2107 Q6AYD4 Cell junction, adherens junction, Cell junction, tight junction, Cell membrane
2108 Q6AYP5 Cell membrane, Cell junction, synapse, Cell junction, synapse, postsynaptic cell membrane
2109 Q6AYR5 Membrane
2110 Q6AYT8 Membrane
2111 Q6AZD4 Mitochondrion inner membrane
2112 Q6DBH0 Endomembrane system
2113 Q6DBW9 Cell membrane, Cell junction
2114 Q6DC58 Membrane
2115 Q6DC66 Membrane
2116 Q6DC75 Membrane
2117 Q6DEL1 Membrane
2118 Q6DG19 Cell membrane
2119 Q6DGA7 Endoplasmic reticulum membrane
2120 Q6DGF5 Golgi apparatus membrane, Endoplasmic reticulum membrane
2121 Q6DH69 Membrane
2122 Q6DHN3 Membrane
2123 Q6DKI7 Cell membrane
2124 Q6DN14 Cytoplasmic vesicle, secretory vesicle, synaptic v

In [44]:
df_WIP.to_csv('./Result_Selected_Organisms_SubCellLoc.csv', index=False)

In [45]:
df_WIP.tail()

,Uniprot_ID,Organism,Protein_name,AH_or_Not,AA_sequence,Prediction,Subcellular_location
2987,O44252,Drosophila melanogaster (Fruit fly),Protein rolling stone,AH,MQLFDDFCKSFNKELQRANFGFAYNRVHLFYRSQWQKDEINTIYLL...,0001111111111100000000000000000000000000000000...,Membrane
2988,O44443,Caenorhabditis elegans,EGF-like domain-containing protein C02B10.3,Non-AH,MTGALCIVLFGVTMVTAERPKIKDTHGNLLVKLSDIPIGSCGDESY...,0000000000000000000000000000000000000000000000...,Membrane
2989,O45145,Caenorhabditis elegans,Alkaline ceramidase,AH,MESSSINRWFEYESGHAWCESAYKYQTLPYVAEFANTCTNLPIIVL...,0000000000000000000000000000000000000000000000...,Membrane
2990,O45150,Caenorhabditis elegans,Serpentine receptor class gamma-30,Non-AH,MLKCHPGFNTKLELLKYGIQFVYFIVGLGFHFAVIKVLHKKWSVYS...,0000000000000000000000000000000000000000000000...,Membrane
2991,O45300,Caenorhabditis elegans,Serpentine receptor class epsilon-13,AH,MSFNISQENEFQTMYIKYFNKTYSIIEGSYNYYLFVFYIQIALIFI...,0000000000000000000000000000000000000000000000...,Membrane
